### Part 2 Web scrapping of Population and Demographics data of New York city from Wikipedia

In [47]:
!pip install beautifulsoup4
!pip install lxml
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation

#### A : POPULATION DATA

Web scrapping of Population data from wikipedia page - https://en.wikipedia.org/wiki/New_York_City

#### Download all the dependencies that is needed.

In [4]:
#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 


from IPython.display import display_html
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library
from bs4 import BeautifulSoup
from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

print('Folium installed')
print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.4.5.2 |       hecda079_0         147 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    certifi-2020.4.5.2         |   py36h9f0ad1d_0         152 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                       

In [56]:
website_url = requests.get('https://en.wikipedia.org/wiki/Demographics_of_New_York_City#Ethnicities_and_enclaves').text
soup = BeautifulSoup(website_url,'lxml')
import csv
table = soup.find('table',{'class':'wikitable sortable'})
#print(soup.prettify())

headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('POPULATION1.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

In [28]:
Pop_data=pd.read_csv('POPULATION1.csv')
Pop_data.drop(Pop_data.columns[[3,4,7,8,9,10,11,12,13,14]], axis=1,inplace=True)
Pop_data
#print('Data downloaded!')

,New York City's five boroughsvte,Jurisdiction,Population,Density,Borough
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.10\n,109.04\n
1,Brooklyn\n,\n Kings\n,"2,559,903\n",70.82\n,183.42\n
2,Manhattan\n,\n New York\n,"1,628,706\n",22.83\n,59.13\n
3,Queens\n,\n Queens\n,"2,253,858\n",108.53\n,281.09\n
4,Staten Island\n,\n Richmond\n,"476,143\n",58.37\n,151.18\n
5,City of New York,"8,336,817",842.343,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","122,284",412
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN


#### Remove whitespaces and rename columns

In [29]:
Pop_data.columns = Pop_data.columns.str.replace(' ', '')
Pop_data.columns = Pop_data.columns.str.replace('\'','')
Pop_data.rename(columns={'Borough':'persons_sq_mi'}, inplace=True)
Pop_data

,NewYorkCitysfiveboroughsvte,Jurisdiction,Population,Density,persons_sq_mi
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.10\n,109.04\n
1,Brooklyn\n,\n Kings\n,"2,559,903\n",70.82\n,183.42\n
2,Manhattan\n,\n New York\n,"1,628,706\n",22.83\n,59.13\n
3,Queens\n,\n Queens\n,"2,253,858\n",108.53\n,281.09\n
4,Staten Island\n,\n Richmond\n,"476,143\n",58.37\n,151.18\n
5,City of New York,"8,336,817",842.343,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","122,284",412
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN


In [30]:
Pop_data.rename(columns = {'NewYorkCitysfiveboroughsvte\n' : 'Borough',
                   'Jurisdiction\n':'County',
                   'Population\n':'Estimate_2019', 
                   'Density\n':'square_mi'}, inplace=True)
Pop_data

,Borough,County,Estimate_2019,square_mi,persons_sq_mi
0,The Bronx\n,\n Bronx\n,"1,418,207\n",42.10\n,109.04\n
1,Brooklyn\n,\n Kings\n,"2,559,903\n",70.82\n,183.42\n
2,Manhattan\n,\n New York\n,"1,628,706\n",22.83\n,59.13\n
3,Queens\n,\n Queens\n,"2,253,858\n",108.53\n,281.09\n
4,Staten Island\n,\n Richmond\n,"476,143\n",58.37\n,151.18\n
5,City of New York,"8,336,817",842.343,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","122,284",412
7,Sources:[14] and see individual borough articl...,NaN,NaN,NaN,NaN


#### Replace newline('\n') from each string from left and right sides

In [31]:
Pop_data['Borough']=Pop_data['Borough'].replace(to_replace='\n', value='', regex=True)
Pop_data['County']=Pop_data['County'].replace(to_replace='\n', value='', regex=True)
Pop_data['Estimate_2019']=Pop_data['Estimate_2019'].replace(to_replace='\n', value='', regex=True)
Pop_data['square_mi']=Pop_data['square_mi'].replace(to_replace='\n', value='', regex=True)
Pop_data['persons_sq_mi']=Pop_data['persons_sq_mi'].replace(to_replace='\n', value='', regex=True)
Pop_data

,Borough,County,Estimate_2019,square_mi,persons_sq_mi
0,The Bronx,Bronx,"1,418,207",42.10,109.04
1,Brooklyn,Kings,"2,559,903",70.82,183.42
2,Manhattan,New York,"1,628,706",22.83,59.13
3,Queens,Queens,"2,253,858",108.53,281.09
4,Staten Island,Richmond,"476,143",58.37,151.18
5,City of New York,"8,336,817",842.343,783.83,"27,547"
6,State of New York,"19,453,561","1,731.910","122,284",412
7,Sources:[14] and see individual borough articles,NaN,NaN,NaN,NaN


#### Shift data in the last two rows

In [33]:
Pop_data.loc[5:,['square_mi','persons_sq_mi']] = Pop_data.loc[2:,['square_mi','persons_sq_mi']].shift(1,axis=1)
Pop_data.loc[5:,['Estimate_2019','square_mi']] = Pop_data.loc[2:,['Estimate_2019','square_mi']].shift(1,axis=1)
Pop_data.loc[5:,['County','Estimate_2019']] = Pop_data.loc[2:,['County','Estimate_2019']].shift(1,axis=1)
Pop_data.loc[5:,['Borough','County']] = Pop_data.loc[2:,['Borough','County']].shift(1,axis=1)
Pop_data

,Borough,County,Estimate_2019,square_mi,persons_sq_mi
0,The Bronx,Bronx,"1,418,207",42.10,109.04
1,Brooklyn,Kings,"2,559,903",70.82,183.42
2,Manhattan,New York,"1,628,706",22.83,59.13
3,Queens,Queens,"2,253,858",108.53,281.09
4,Staten Island,Richmond,"476,143",58.37,151.18
5,NaN,City of New York,"8,336,817",842.343,783.83
6,NaN,State of New York,"19,453,561","1,731.910","122,284"
7,NaN,Sources:[14] and see individual borough articles,NaN,NaN,NaN


#### Remove 'NAN'

In [34]:
Pop_data = Pop_data.fillna('')
Pop_data

,Borough,County,Estimate_2019,square_mi,persons_sq_mi
0,The Bronx,Bronx,"1,418,207",42.10,109.04
1,Brooklyn,Kings,"2,559,903",70.82,183.42
2,Manhattan,New York,"1,628,706",22.83,59.13
3,Queens,Queens,"2,253,858",108.53,281.09
4,Staten Island,Richmond,"476,143",58.37,151.18
5,,City of New York,"8,336,817",842.343,783.83
6,,State of New York,"19,453,561","1,731.910","122,284"
7,,Sources:[14] and see individual borough articles,,,


#### Drop the last row

In [36]:
i = Pop_data[((Pop_data.County == 'Sources:[14] and see individual borough articles'))].index
Pop_data.drop(i)

,Borough,County,Estimate_2019,square_mi,persons_sq_mi
0,The Bronx,Bronx,"1,418,207",42.10,109.04
1,Brooklyn,Kings,"2,559,903",70.82,183.42
2,Manhattan,New York,"1,628,706",22.83,59.13
3,Queens,Queens,"2,253,858",108.53,281.09
4,Staten Island,Richmond,"476,143",58.37,151.18
5,,City of New York,"8,336,817",842.343,783.83
6,,State of New York,"19,453,561","1,731.910","122,284"


#### Save dataframe as csv file

In [37]:
Pop_data.to_csv('POPULATION.csv',index=False)

#### B : DEMOGRAPHICS DATA

We will web scrap Demographics data from wikipedia page - https://en.wikipedia.org/wiki/New_York_City

#### Web scrapping of Demographics data from wikipedia page using BeautifulSoup.

Beautiful Soup is a Python package for parsing HTML and XML documents (including having malformed markup, i.e. non-closed tags, so named after tag soup). It creates a parse tree for parsed pages that can be used to extract data from HTML, which is useful for web scraping.

In [6]:
website_url = requests.get('https://en.wikipedia.org/wiki/Demographic_history_of_New_York_City').text
soup = BeautifulSoup(website_url,'lxml')
table = soup.find('table',{'class':'wikitable'})
#print(soup.prettify())
import csv
headers = [header.text for header in table.find_all('th')]

table_rows = table.find_all('tr')        
rows = []
for row in table_rows:
   td = row.find_all('td')
   row = [row.text for row in td]
   rows.append(row)

with open('DEMO.csv', 'w') as f:
   writer = csv.writer(f)
   writer.writerow(headers)
   writer.writerows(row for row in rows if row)

#### Load data from CSV

In [30]:
Demo_data=pd.read_csv('DEMO.csv')
print('Data downloaded!')

Data downloaded!


In [31]:
Demo_data

,Year,Population,White(includes White Hispanics),%W,Non-Hispanic Whites,%ANG,Black,%B,Asian,%A,Other orMixed,%O/M,Hispanic/Latino,%H/L,Foreignborn,%FB
0,1900,"3,437,202","3,369,898",98.04,NaN,NaN,"60,666",1.76,"6,607",0.19,31,0.00,NaN,NaN,"1,270,080",36.95
1,1910,"4,766,883","4,669,162",97.95,NaN,NaN,"91,709",1.92,"5,669",0.12,343,0.01,NaN,NaN,"1,944,357",40.79
2,1920,"5,620,048","5,459,463",97.14,NaN,NaN,"152,467",2.71,"7,969",0.14,149,0.00,NaN,NaN,"2,028,160",36.09
3,1930,"6,930,446","6,589,377",95.08,NaN,NaN,"327,706",4.73,"12,972",0.19,391,0.01,NaN,NaN,"2,358,686",34.03
4,1940,"7,454,995","6,977,501",93.59,"6,856,586",91.97,"458,444",6.15,"17,986",0.24,"1,064",0.01,"120,915",1.62,"2,138,657",28.69
5,1950,"7,891,957","7,116,441",90.17,NaN,NaN,"747,608",9.47,"21,441",0.27,"6,467",0.08,NaN,NaN,"1,784,206",22.61
6,1960,"7,781,984","6,640,662",85.33,NaN,NaN,"1,087,931",13.98,"43,103",0.55,"10,288",0.13,NaN,NaN,"1,558,690",20.03
7,1970,"7,894,862","6,048,841",76.62,"4,969,749",62.95,"1,668,115",21.13,"94,499",1.20,"83,407",1.06,"1,278,630",16.20,"1,437,058",18.20
8,1980,"7,071,639","4,294,075",60.72,"3,668,945",51.88,"1,784,337",25.23,"231,501",3.27,"761,762",10.77,"1,406,024",19.88,"1,670,199",23.62
9,1990,"7,322,564","3,827,088",52.26,"3,163,125",43.20,"2,102,512",28.71,"512,719",7.00,"880,245",12.02,"1,783,511",24.36,"2,082,931",28.45


In [32]:
Demo_data.to_csv('DEMOGRAPHICS.csv',index=False)

In [33]:
Demo_data=Demo_data.iloc[10:,]
Demo_data

,Year,Population,White(includes White Hispanics),%W,Non-Hispanic Whites,%ANG,Black,%B,Asian,%A,Other orMixed,%O/M,Hispanic/Latino,%H/L,Foreignborn,%FB
10,2000,"8,008,278","3,576,385",44.66,"2,801,267",34.98,"2,129,762",26.59,"792,477",9.90,"1,509,654",18.85,"2,160,554",26.98,"2,871,032",35.85
11,2010,"8,175,133","3,597,341",44.00,"2,722,904",33.31,"2,088,510",25.55,"1,043,535",12.77,"1,445,747",17.68,"2,336,076",28.58,"3,066,599",37.51


In [34]:
Demo_data.shape

(2, 16)

In [35]:
Demo_data.drop(Demo_data.columns[[2,4,6,8,10,12,14]], axis=1,inplace=True)
Demo_data

,Year,Population,%W,%ANG,%B,%A,%O/M,%H/L,%FB
10,2000,"8,008,278",44.66,34.98,26.59,9.90,18.85,26.98,35.85
11,2010,"8,175,133",44.00,33.31,25.55,12.77,17.68,28.58,37.51


In [44]:
Demo_data.rename(columns = {'%W' : 'White(includes White Hispanics)%',
                   '%ANG':'Non-Hispanic Whites %',
                   '%B':'Black %', 
                   '%A':'Asian %',
                   '%O/M':'Other orMixed %',
                   '%H/L':'Hispanic/Latino %', 
                   '%FB':'Foreignborn %'}, inplace=True)
Demo_data

,Year,Population,White(includes White Hispanics)%,Non-Hispanic Whites %,Black %,Asian %,Other orMixed %,Hispanic/Latino %,%FB
10,2000,"8,008,278",44.66,34.98,26.59,9.90,18.85,26.98,35.85
11,2010,"8,175,133",44.00,33.31,25.55,12.77,17.68,28.58,37.51


In [48]:
Demo_data.transpose()
Demo_data

,Year,Population,White(includes White Hispanics)%,Non-Hispanic Whites %,Black %,Asian %,Other orMixed %,Hispanic/Latino %,%FB
10,2000,"8,008,278",44.66,34.98,26.59,9.90,18.85,26.98,35.85
11,2010,"8,175,133",44.00,33.31,25.55,12.77,17.68,28.58,37.51


In [52]:
Demo_data.drop("Population", axis=1,inplace=True)
Demo_data

,Year,White(includes White Hispanics)%,Non-Hispanic Whites %,Black %,Asian %,Other orMixed %,Hispanic/Latino %,%FB
10,2000,44.66,34.98,26.59,9.90,18.85,26.98,35.85
11,2010,44.00,33.31,25.55,12.77,17.68,28.58,37.51


In [55]:
Demo_data.to_csv(r'C:\Users\crahate\Desktop\DEMOGRAPHICS.csv',index=False,header=True)